In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.4 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")

In [3]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [7]:
data = pd.read_csv('/home/apprenant/Documents/01projet_python/DevIA_Roubaix/NLP/data/cleaned_data.csv')
texts = data['Text'].tolist()
labels = data['Emotion'].tolist()

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)

model_name = "michellejieli/emotion_text_classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels).tolist()
test_labels_encoded = label_encoder.transform(test_labels).tolist()

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels_encoded
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels_encoded
))

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.fit(train_dataset.shuffle(1000).batch(16), epochs=3)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Epoch 1/3
1073/1073 [==============================] - 209s 171ms/step - loss: 0.3373 - accuracy: 0.8841
Epoch 2/3
1073/1073 [==============================] - 168s 156ms/step - loss: 0.1625 - accuracy: 0.9323
Epoch 3/3
1073/1073 [==============================] - 167s 156ms/step - loss: 0.1292 - accuracy: 0.9404


In [8]:
model.evaluate(test_dataset.batch(64))


68/68 [==============================] - 12s 149ms/step - loss: 0.1723 - accuracy: 0.9306


[0.17234298586845398, 0.930568516254425]

In [9]:
model.save_pretrained("emotion_preds")


In [11]:
tokenizer.save_pretrained("emotion_preds")


('emotion_preds/tokenizer_config.json',
 'emotion_preds/special_tokens_map.json',
 'emotion_preds/vocab.json',
 'emotion_preds/merges.txt',
 'emotion_preds/added_tokens.json',
 'emotion_preds/tokenizer.json')